This notebook trains an RF and NB agent on the extended data.

A separate dataset would be used to train the RL agent.

The downside to this method is that the RL method relies heavily on the accuracy of the RF and NB models and would be limited by their accuracy.

In [1]:
#module_dir = "/Users/teliov/TUD/symcat-to-synthea/output/module_ai_med_extended"
module_dir = "/home/gzr/文档/medvice/thesis/data/NLICE"

In [2]:
from thesislib.utils.ml import process
from sklearn.model_selection import StratifiedShuffleSplit

In [34]:
symptom_map, condition_map = process.get_symptom_condition_map(module_dir)
print(symptom_map)

{'aef44a382653203b3a250f04d5d4ab93b2ac410d75cbfa87b413569f': 'aura', '0cfc2a9e603e4df4a54c49478bc65b8daaf5b5844334a29d63e59cef': 'altered_vision', '88e3734465cfa73e79558a79b395d4492318ce1decdecd7409af6134': 'weakness', 'cde2666177368527bc5740613160ac5ccde486ee50bfbd7c48434099': 'altered_mental_state', 'f108db3f6fefb67927dda21aaad2f8887ad77e8fffe3e0915ac2681a': 'focal_neurological_symptoms', 'ab13eeecfc96af0c8a78818438af23c643522e51c04ba9c217b868b3': 'stiffness', '7d063801e12b750d1176a339c4a5eebf4a39554e7502734e1355ec3d': 'vomiting', '796c7f05c32836db0f3cccfc7ae8e96e2a7157455e39d2d45ebbea3b': 'pain', 'a1b4c06c3ea6ab83d933b1980b6738a683aec3825b3f73a390c0c741': 'nausea', '2fc1b23c0e13e3aeb134ee121d9d3343aaa53bf4b7820fb0873ff69e': 'slow_healing', '41eaa97fe501696ef3c3ab5402142d2c82a2704802f5962c93b3a82f': 'eruption', '2be670da9bd827b524a664f0dc9b119d02e005580da6844f2393b71a': 'altered_sleep', 'a19c9e3866a4b0dab527510697c3d291dc1cb0ac15ed9b58983ab99c': 'throbbing', '87835ab9d2d63526b21f44ff

In [3]:
import pathlib

/home/gzr/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
#ata_dir = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/extended"
data_dir = "/home/gzr/文档/medvice/thesis/data/basic"
pathlib.Path(data_dir).mkdir(exist_ok=True, parents=True)

In [9]:
import json
import os

In [38]:
symptom_map_file = os.path.join(data_dir, "symptom_db.json")
with open(symptom_map_file, "w") as fp:
    json.dump(symptom_map, fp, indent=4)

In [39]:
condition_map_file = os.path.join(data_dir, "condition_db.json")
with open(condition_map_file, "w") as fp:
    json.dump(condition_map, fp, indent=4)

In [4]:
from thesislib.utils.ml import process
from thesislib.utils.ml import runners, models
import pandas as pd

In [5]:
#data_csv = "/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/ai/output_med_ai_ext/symptoms/csv/symptoms.csv"
data_csv = "/home/gzr/文档/medvice/thesis/data/nlice100k/symptoms/csv/symptoms.csv"

In [71]:
def split_data(symptom_file, output_path, use_headers=False, train_split=0.8):
    symptom_columns = ['PATIENT', 'GENDER', 'RACE', 'ETHNICITY', 'AGE_BEGIN', 'AGE_END',
                       'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS']

    pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

    if use_headers:
        df = pd.read_csv(symptom_file, names=symptom_columns)
    else:
        df = pd.read_csv(symptom_file,sep = "\t")
    df.index.name = "Index" 
    labels = df["PATHOLOGY"].drop(0)
    datas = df.drop(0)
    print(labels)
    #print(labels.value_counts())
    splitter = StratifiedShuffleSplit(1, train_size=train_split)
    train_index = None
    test_index = None
    for tr_idx, tst_index in splitter.split(datas, labels):
        train_index = tr_idx
        test_index = tst_index
        break

    train_df = df.iloc[train_index]
    test_df = df.iloc[test_index]

    train_op = os.path.join(output_path, "train.csv")
    test_op = os.path.join(output_path, "test.csv")
    train_df.to_csv(train_op)
    test_df.to_csv(test_op)
    return train_op, test_op


In [72]:
op_data_dir = os.path.join(data_dir, "data")
# split into train and test
train_file, test_file = split_data(data_csv, op_data_dir,True,train_split=0.9)

parsed_data_dir = os.path.join(op_data_dir, "parsed")

/home/gzr/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Index
1                                       Giardiasis
2                           Lumbar spinal stenosis
3                          Pulmonary edema  oldest
4                                   Osteoarthritis
5                             Bacterial meningitis
                            ...                   
394686                      Carpal tunnel syndrome
394687                                      Asthma
394688                        Pulmonary edema  old
394689                     Pulmonary edema  oldest
394690    Viral meningitis  varicella zoster virus
Name: PATHOLOGY, Length: 394690, dtype: object
COPD                                        13471
Gout                                        13418
Rheumatoid arthritis  feminine              12980
IBS (Constipation)                          12870
Hernia Nuclei Pulposi                       12738
Pulmonary edema  oldest                     12660
Asthma                                      12655
Appendicitis                        

In [73]:
import pandas as pd
import numpy as np

/home/gzr/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
#parse the train set
RACE_CODE = {'white': 0, 'black':1, 'asian':2, 'native':3, 'other':4}
def parse_data_nlice_adv(
        filepath,
        conditions_db_json,
        symptoms_db_json,
        output_path,
        body_parts_json,
        excitation_enc_json,
        frequency_enc_json,
        nature_enc_json,
        vas_enc_json,
        onset_json,
        duration_json
        ):
    pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)
    with open(symptoms_db_json) as fp:
        symptoms_db = json.load(fp)

    with open(conditions_db_json) as fp:
        conditions_db = json.load(fp)

    with open(body_parts_json) as fp:
        body_parts = json.load(fp)

    with open(excitation_enc_json) as fp:
        excitation_enc = json.load(fp)

    with open(frequency_enc_json) as fp:
        frequency_enc = json.load(fp)

    with open(nature_enc_json) as fp:
        nature_enc = json.load(fp)

    with open(vas_enc_json) as fp:
        vas_enc = json.load(fp)

    with open(onset_json) as fp:
        onset_enc = json.load(fp)

    with open(duration_json) as fp:
        duration_enc = json.load(fp)

    usecols = ['GENDER', 'RACE', 'AGE_BEGIN', 'PATHOLOGY', 'NUM_SYMPTOMS', 'SYMPTOMS']

    df = pd.read_csv(filepath, usecols=usecols)
    filename = filepath.split("/")[-1]

    # drop the guys that have no symptoms
    #f['NUM_SYMPTOMS'] = df['NUM_SYMPTOMS'].astype(int)
    #df = df[df['NUM_SYMPTOMS']> 0]
    df['LABEL'] = df.PATHOLOGY.apply(lambda v: conditions_db.get(v))
    df['RACE'] = df.RACE.apply(lambda v: RACE_CODE.get(v))
    df['GENDER'] = df.GENDER.apply(lambda gender: 0 if gender == 'F' else 1)
    df = df.rename(columns={'AGE_BEGIN': 'AGE'})
    print(df.SYMPTOMS)
    df['SYMPTOMS'] = df.SYMPTOMS.apply(
        transform_symptoms_nlice_adv,
        symptom_db=symptoms_db,
        body_parts=body_parts,
        excitation_enc=excitation_enc,
        frequency_enc=frequency_enc,
        nature_enc=nature_enc,
        vas_enc=vas_enc,
        onset_enc = onset_enc,
        duration_enc = duration_enc
    )

    ordered_keys = ['LABEL', 'GENDER', 'RACE', 'AGE', 'SYMPTOMS']
    df = df[ordered_keys]
    df.index.name = "Index"
    output_file = os.path.join(output_path, "%s_sparse.csv" % filename)
    df.to_csv(output_file)

    return output_file

def transform_symptoms_nlice_adv(
        symptom_str,
        symptom_db,
        body_parts,
        excitation_enc,
        frequency_enc,
        nature_enc,
        vas_enc,
        onset_enc,
        duration_enc
):  
    print(symptom_str)
    symptom_list = symptom_str.split(";")
    transformed_symptoms = []
    for _symp_def in symptom_list:
        print(_symp_def)
        sym_list = _symp_def.split(":")
        if(len(sym_list)==9):
            _symptom, _nature, _location, _intensity, _duration, _onset, _exciation, _frequency, _ = _symp_def.split(":")

            _symptom_idx = symptom_db[_symptom] * 8

            _nature_idx = _symptom_idx + 1
            _nature_val = 1 if _nature == "" or _nature == "other" else nature_enc.get(_nature)

            _location_idx = _symptom_idx + 2
            _location_val = 1 if _location == "" or _location == "other" else body_parts.get(_location)

            _intensity_idx = _symptom_idx + 3
            _intensity_val = 1 if _intensity == "" else vas_enc.get(_intensity)

            _duration_idx = _symptom_idx + 4
            _duration_val = 0 if _duration == "" else duration_enc.get(_duration)

            _onset_idx = _symptom_idx + 5
            _onset_val = 0 if _onset == "" else onset_enc.get(_onset)

            _excitation_idx = _symptom_idx + 6
            _excitation_val = 1 if _exciation == "" else excitation_enc.get(_exciation)

            _frequency_idx = _symptom_idx + 7
            _frequency_val = 1 if _frequency == "" else frequency_enc.get(_frequency)

            to_transform = [
                "|".join([str(_symptom_idx), "1"]),
                "|".join([str(_nature_idx), str(_nature_val)]),
                "|".join([str(_location_idx), str(_location_val)]),
                "|".join([str(_intensity_idx), str(_intensity_val)]),
                "|".join([str(_excitation_idx), str(_excitation_val)]),
                "|".join([str(_frequency_idx), str(_frequency_val)])
            ]

            if _duration_val != 0:
                to_transform.append(
                    "|".join([str(_duration_idx), str(_duration_val)])
                )

            if _onset_val != 0:
                to_transform.append(
                    "|".join([str(_onset_idx), str(_onset_val)]),
                )

            transformed_str = ";".join(to_transform)

            transformed_symptoms.append(transformed_str)

    return ";".join(transformed_symptoms)

In [38]:
# parse the train set and let's train
train_file = "/home/gzr/文档/medvice/thesis/data/basic/data/train.csv"
test_file = "/home/gzr/文档/medvice/thesis/data/basic/data/test.csv"
conditions_db_file = "/home/gzr/文档/medvice/thesis/data/basic/conditions_db.json"
symptoms_db_file = "/home/gzr/文档/medvice/thesis/data/basic/symptoms_db.json"
body_parts_file = "/home/gzr/文档/medvice/thesis/data/basic/body-parts-enc.json"
excitation_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/excitation_encoding.json"
frequency_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/frequency_encoding.json"
nature_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/nature_encoding.json"
vas_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/vas_encoding.json"
onset_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/onset_encoding.json"
duration_enc_file = "/home/gzr/文档/medvice/thesis/data/basic/duration_encoding.json"
parsed_data_dir = "/home/gzr/文档/medvice/thesis/data/basic/data/parsed"
        # excitation_enc_json,
        # frequency_enc_json,
        # nature_enc_json,
        # vas_enc_json
parsed_train = parse_data_nlice_adv(
    train_file,
    conditions_db_file,
    symptoms_db_file,
    parsed_data_dir,
    body_parts_file,
    excitation_enc_file,
    frequency_enc_file,
    nature_enc_file,
    vas_enc_file,
    onset_enc_file,
    duration_enc_file
)

/home/gzr/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


0              Pain_relief::::::Sitting/bending_forward::29
1         Vomitting::::0_-_2_days::::29;Altered_stool:Mu...
2         Vomiting::::::::46;Altered_stool:Diarrhea:::::...
3                                            Pain::::::::42
4         Focal_neurological_symptoms::::::::27;Pain::::...
                                ...                        
355216    Difficulty_walking:::::::Periodic:42;Eruption:...
355217    Tightness::::::::33;Altered_breathing:Wheezing...
355218    Tightness::::::::33;Altered_breathing:Wheezing...
355219    Pain::::::::45;Vomiting::::::::28;Altered_stoo...
355220    Stiffness::::::Morning::49;Fever::::::::27;Dry...
Name: SYMPTOMS, Length: 355221, dtype: object
Pain_relief::::::Sitting/bending_forward::29
Pain_relief::::::Sitting/bending_forward::29


TypeError: object of type 'bool' has no len()

In [ ]:
# parse the test set for evaluation
# parse the train set and let's train
parsed_test = process.parse_data(
    test_file,
    condition_map_file,
    symptom_map_file,
    parsed_data_dir
)

In [ ]:
# train with RF and then with NB
rf_dir = os.path.join(op_data_dir, "output/rf")
rfparams = models.RFParams()
rfparams.n_estimators = 200
rfparams.max_depth = None

pathlib.Path(rf_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
run_ok = runners.train_ai_med_rf(
    parsed_train,
    symptom_map_file,
    rf_dir,
    rfparams,
    "Basic AI-MED Run",
    "local-pc"
)

In [ ]:
# train NB
nb_dir = os.path.join(op_data_dir, "output/nb")

run_ok = runners.train_ai_med_nb(
    parsed_train,
    symptom_map_file,
    nb_dir,
    "Basic AI-MED Run",
    "local-pc"
)

In [ ]:
# we'll evaluate on the unseen data ..

In [ ]:
import joblib

In [ ]:
nb_data = joblib.load("/Users/teliov/TUD/Thesis/Medvice/Notebooks/data/06_18_nlice_plus/nlice-adv/data/output/nb_100k/nb_serialized_sparse.joblib")

In [ ]:
nb_clf = nb_data.get('clf')

In [ ]:
vec=[[1,0,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,126,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]

In [ ]:
import numpy as np

In [ ]:
vec = np.array(vec)

In [ ]:
from scipy.sparse import csc_matrix

In [ ]:
vec = csc_matrix(vec)

In [ ]:
res = nb_clf.predict_proba(vec)

In [ ]:
res

array([[4.92455255e-26, 2.42705705e-19, 4.85147978e-07, 7.40938886e-12,
        9.99996200e-01, 8.45675481e-36, 3.31381133e-06, 8.67802527e-10,
        1.27774149e-40, 1.13143592e-44, 1.46078507e-12, 1.21781484e-22,
        4.11544543e-20, 2.53092584e-27]])